# #  Seabed Mobility [2019, 2020, 2023, 2024]

#### Summary 
The SummaryThe code is used for seabed morphology and mobility analysis by comparing elevation data from different years (2019, 2020, 2023, 2024, 2025). It calculates differences between raster datasets, applies corrections, clips the results to buffer zones around wind turbine generator (WTG) locations, and computes zonal statistics. These statistics are then exported to Excel and CSV files for further analysis and reporting. Main Steps in the Code:
- Environment Setup
- Raster Difference Calculation and Clipping:
- Calculates statistical metrics (Min, Max, Mean, Percentile 90) for each clipped raster within the buffer zone.
- Merges zonal statistics from multiple rasters into a single Excel file and then converts it into CSV.
- Joins the CSV data with the WTG shapefile based on common columns and creates a new shapefile with the results.


## Posible  Future Changes: 

**1) New Raster Pair Definition:** raster_pairs = [...
            (raster_2026, raster_2019, "26_19.tif", 0.9), ... ]*
            
**2) Buffer distance:**    *buffer_distance = "100 meters"*     

**3) The code correctly applies mean corrections 0.9--> 1.5:**
*raster_pairs = [...(raster_2026, raster_2019, "26_19.tif", 1.5), ...]*
            
**4) The fields MAX, MIN, MEAN, and PCT90 are hardcoded in the calculate_zonal_statistics function. e.g. 75th percentile directly**
*fields_to_keep = ["MAX", "MIN", "MEAN", "PCT75", abs_mean_field_name]*

# # Delivery 1st - Min Max Abs Mean PCT90

In [1]:
import arcpy
import numpy as np
import os
import pandas as pd
from openpyxl import Workbook

# Set environment
arcpy.env.overwriteOutput = True
workspace = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin"
arcpy.env.workspace = workspace

# Input rasters (update file paths once 2024 data is available)
raster_2019 = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\SITEINV\20240206_from_WTP_004502963-03\004502963-03-MASP - GIS Data - Task 1 - Seabed Morphology and Mobility\RWE_Thor_Task1_v0.3.gdb\SN2019_011_R_DTM_MBES_100CM"
raster_2020 = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\SITEINV\20240206_from_WTP_004502963-03\004502963-03-MASP - GIS Data - Task 1 - Seabed Morphology and Mobility\RWE_Thor_Task1_v0.3.gdb\MMT_628_GU_OWF_T50R_DTU15_MSL_DTM_100CM"
raster_2023 = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\SITEINV\SITEINV.gdb\DK_THO_Fugro_MBES_2023_DEM_Mean_1pt00m_ra_UTM32N_EXT"
raster_2024 = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\SITEINV\20240924_Bathy\F248460_RWE_Thor_1m_MSL\F248460_RWE_Thor_1m_MSL.tif"

wtg = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\WTG\WTG.gdb\DK_THO_Layout_pt_UTM32N_v7"


# List of output difference raster names and correction values
raster_pairs = [
    (raster_2024, raster_2019, "24_19.tif", 0),     # 2024 - 2019
    (raster_2024, raster_2023, "24_23.tif", -0.09), # 2024 - 2023
    (raster_2023, raster_2019, "23_19.tif", 0.11),  # 2023 - 2019 (+0.11)
    (raster_2023, raster_2020, "23_20.tif", 0.09),  # 2023 - 2020 (+0.09)
    (raster_2020, raster_2019, "20_19.tif", 0)      # 2020 - 2019 (no correction)
]

# Buffer parameters
buffer_distance = "200 meters"  # Example buffer size

### A) Calculate difference rasters B) clip them to the buffer zone C) compute statistics

In [4]:
# Function to create a buffer around the WTG locations
def create_wtg_buffer(wtg_fc, buffer_distance, output_buffer):
    arcpy.Buffer_analysis(wtg_fc, output_buffer, buffer_distance)

# Combined function to calculate raster difference and clip to the buffer zone
def calculate_and_clip_difference_raster(raster1, raster2, buffer_fc, output_name, mean_correction=0):
    # Calculate raster difference
    diff_raster = arcpy.sa.Raster(raster1) - arcpy.sa.Raster(raster2)
    
    # Apply mean correction if provided
    if mean_correction != 0:
        diff_raster += mean_correction
    
    # Clip the difference raster to the buffer zone
    output_clipped_raster = os.path.join(workspace, output_name)
    arcpy.Clip_management(diff_raster, "#", output_clipped_raster, buffer_fc, "NoData", "ClippingGeometry")
    print(f"Raster clipped to buffer zone: {output_clipped_raster}")
    
    
    return output_clipped_raster

# Create buffer around WTG locations
wtg_buffer = os.path.join(workspace, "WTG_Buffer_200m.shp")
create_wtg_buffer(wtg, buffer_distance, wtg_buffer)

# Process rasters
for r1, r2, output_name, correction in raster_pairs:
    if arcpy.Exists(r1) and arcpy.Exists(r2):  # Only process if both rasters exist
        print(f"Processing difference and clipping for {output_name}...")
        calculate_and_clip_difference_raster(r1, r2, wtg_buffer, output_name, correction)
        


Processing difference and clipping for 24_19.tif...
Raster clipped to buffer zone: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\24_19.tif
Processing difference and clipping for 24_23.tif...
Raster clipped to buffer zone: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\24_23.tif
Processing difference and clipping for 23_19.tif...
Raster clipped to buffer zone: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\23_19.tif
Processing difference and clipping for 23_20.tif...
Raster clipped to buffer zone: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\23_20.tif
Processing difference and clipping for 20_19.tif...
Raster clipped to buffer zone: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\20_19.tif


### B) Zonal statistics

In [6]:
# Function to calculate zonal statistics
def calculate_zonal_statistics(diff_raster, buffer_fc, zone_field, output_table, raster_name_prefix):

    # Shorten raster name prefix to a maximum of 6 characters for field name compliance
    raster_name_prefix = raster_name_prefix[:6]  # Ensures prefix is max 6 characters

    # Create Zonal Statistics tables for Max, Min, and Mean
    arcpy.sa.ZonalStatisticsAsTable(buffer_fc, zone_field, diff_raster, output_table, "DATA", "All")
    
    # Add the field with the new shorter name for Abs Mean (keeping it under 10 characters)
    abs_mean_field_name = f"Abs_{raster_name_prefix}"
    arcpy.AddField_management(output_table, abs_mean_field_name, "DOUBLE")
    
    # Absolute value raster for mean of magnitude
    abs_diff_raster = arcpy.sa.Abs(diff_raster)
    
    # Calculate mean of magnitude (average of absolute values)
    abs_table = os.path.join(workspace, "abs" + os.path.basename(output_table))
    arcpy.sa.ZonalStatisticsAsTable(buffer_fc, zone_field, abs_diff_raster, abs_table, "DATA", "MEAN")
    
    # Ensure the correct field name is referenced for Abs Mean calculation
    with arcpy.da.UpdateCursor(output_table, [zone_field, abs_mean_field_name]) as cursor:
        with arcpy.da.SearchCursor(abs_table, [zone_field, "MEAN"]) as abs_cursor:
            abs_dict = {row[0]: row[1] for row in abs_cursor}  # Create a dictionary of zone ID and Abs Mean values
            for row in cursor:
                zone_id = row[0]
                if zone_id in abs_dict:
                    row[1] = abs_dict[zone_id]
                cursor.updateRow(row)
    
    # Read the output table into a DataFrame (only keep Max, Min, Mean, and Abs Mean fields)
    fields_to_keep = ["MAX", "MIN", "MEAN", "PCT90", abs_mean_field_name]
    df = pd.DataFrame.from_records(
        arcpy.da.TableToNumPyArray(output_table, [zone_field] + fields_to_keep)
    )  # Convert DBF to NumPy array and then to DataFrame
    
    # Rename columns with raster name prefix (keeping the fields under the 10 character limit)
    df.columns = [f"{col[:4]}_{raster_name_prefix}" if col != zone_field else col for col in df.columns] 
    
    # Change column names starting with 'Lay' to start with 'WTG'
    df.columns = [col.replace('Layout_ID1', 'WTG') if col.startswith('Lay') else col for col in df.columns]
    
    # Round all numeric columns to 3 decimal places
    numeric_cols = df.select_dtypes(include='number').columns  # Get numeric columns
    df[numeric_cols] = df[numeric_cols].round(2)  # Round to 2 decimal places
    
    # Export the DataFrame to an Excel file
    output_excel = os.path.join(workspace, os.path.basename(output_table).replace(".dbf", ".xlsx"))
    df.to_excel(output_excel, index=False, engine='openpyxl')

    print(f"Zonal statistics saved to {output_excel}")

# Zone field for WTG locations (use the field that represents unique WTG identifiers)
zone_field = "Layout_ID1"


# Initialize an empty DataFrame for merging
merged_df = pd.DataFrame()

# Iterate through raster pairs and calculate zonal statistics and join excel 
for r1, r2, output_name, correction in raster_pairs:
    if arcpy.Exists(r1) and arcpy.Exists(r2):  # Only process if both rasters exist
        print(f"Calculating zonal statistics for {output_name}...")
        output_table = os.path.join(workspace, os.path.basename(output_name).replace(".tif", ".dbf"))
        raster_name_prefix = os.path.splitext(os.path.basename(output_name))[0]  # Extract raster name for prefixing
        calculate_zonal_statistics(output_name, wtg_buffer, zone_field, output_table, raster_name_prefix)
        output_excel = os.path.join(workspace, os.path.basename(output_name).replace(".tif", ".xlsx"))
        df = pd.read_excel(output_excel)
         # Merge with the existing DataFrame on 'WTG' column
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on='WTG', how='outer')
        
# Save the merged DataFrame to a new Excel file
merged_output_excel = os.path.join(workspace, "merged_zonal_statistics.xlsx")
merged_df.to_excel(merged_output_excel, index=False, engine='openpyxl')

print(f"Merged Excel file saved as: {merged_output_excel}")


Calculating zonal statistics for 24_19.tif...
Zonal statistics saved to \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\24_19.xlsx
Calculating zonal statistics for 24_23.tif...
Zonal statistics saved to \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\24_23.xlsx
Calculating zonal statistics for 23_19.tif...
Zonal statistics saved to \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\23_19.xlsx
Calculating zonal statistics for 23_20.tif...
Zonal statistics saved to \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\23_20.xlsx
Calculating zonal statistics for 20_19.tif...
Zonal statistics saved to \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\20_19.xlsx
Merged Excel file saved as: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_Justi

### C) Join Merged Stat excel file with WTG buffer shp

In [ ]:
temp_csv_path = os.path.join(workspace, "merged_statistics.csv")

# Load the merged Excel into a DataFrame and convert to CSV
merged_df = pd.read_excel(merged_output_excel)

# Replace 0 in numeric columns with NaN
merged_df.loc[:, merged_df.select_dtypes(include='number').columns] = merged_df.select_dtypes(include='number').replace(0, np.nan)
merged_df.to_csv(temp_csv_path, index=False)

shapefile_join_column = "Layout_ID1"  # Column name in the shapefile
csv_join_column = "WTG"  # Column name in the CSV

# Add join
arcpy.management.JoinField(wtg_buffer, shapefile_join_column, temp_csv_path, csv_join_column)
arcpy.management.CopyFeatures(wtg_buffer, r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\STAT_backup.shp")


# # Delivery 2nd - Get excel files with All cell values at each WTG Locations 

In [28]:
buffer_shapefile = r'\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\WTG_Buffer_200m.shp'

# Create a feature layer from the feature class
arcpy.MakeFeatureLayer_management(buffer_shapefile, "buffer_layer")

# Select the points that match the ID
arcpy.SelectLayerByAttribute_management("buffer_layer", "NEW_SELECTION", "Layout_ID1 = 'T16' OR Layout_ID1 = 'T30'")

# Optional: Save the selected points to a new feature class
output_selected_points = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\selected_T16_T30.shp"
arcpy.CopyFeatures_management("buffer_layer", output_selected_points)


<Result '\\\\WM20ocqu46ph01\\WF_Projects\\DK_THO\\4_OUTPUT\\SITEINV\\20241010_SeabedMobility_JustinPeterLarkin\\selected_T16_T30.shp'>

### 1) Iterate through selected feature class points.

In [68]:

buffer_shapefile = r'\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\WTG_Buffer_200m.shp'

# Create a feature layer from the feature class
arcpy.MakeFeatureLayer_management(buffer_shapefile, "buffer_layer")

# Select the points that match the ID
arcpy.SelectLayerByAttribute_management("buffer_layer", "NEW_SELECTION", "Layout_ID1 = 'T16' OR Layout_ID1 = 'T30'")

# Optional: Save the selected points to a new feature class
output_selected_points = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\selected_T16_T30.shp"
arcpy.CopyFeatures_management("buffer_layer", output_selected_points)


<Result '\\\\WM20ocqu46ph01\\WF_Projects\\DK_THO\\4_OUTPUT\\SITEINV\\20241010_SeabedMobility_JustinPeterLarkin\\selected_T16_T30.shp'>

### 2) Clip the raster around each point using a buffer, Extract cliped raster, Calculate statistics and writ all cell valuest to excel file

In [69]:

# Paths to input feature class
feature_class = output_selected_points
# Create a feature layer from the feature class
arcpy.MakeFeatureLayer_management(feature_class, "points_layer")


# Output directory for Excel files and clipped raster exports
output_directory = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\Selected_Buffer"

# Create a feature layer from the feature class
# arcpy.MakeFeatureLayer_management(feature_class, "points_layer")

# Iterate through each raster pair
for raster1, raster2, output_name, correction in raster_pairs:
    print(output_name)

    # Extract suffix from the raster output name (e.g., "23_20" from "23_20.tif")
    suffix = os.path.splitext(output_name)[0]

    # Now, iterate through each point to extract values from the difference raster
    with arcpy.da.SearchCursor("points_layer", ["Layout_ID1", "SHAPE@"]) as cursor:
        for row in cursor:
            point_id = row[0]  # Assuming 'Layout_ID1' contains the point ID
            point_geom = row[1]  # SHAPE@ returns the geometry object of the point

            try:
                # Clip the difference raster using the point geometry (or buffer if needed)
                clipped_raster = arcpy.sa.ExtractByMask(output_name, point_geom)
    
                # Export the clipped raster to a file
                clipped_raster_path = os.path.join(output_directory, f"clipped_raster_{point_id}_{suffix}.tif")
                clipped_raster.save(clipped_raster_path)
    
                print(f"Exported clipped raster for point ID {point_id} to {clipped_raster_path}")
    
                # Convert the clipped raster to a NumPy array
                arr = arcpy.RasterToNumPyArray(clipped_raster, nodata_to_value=np.nan)
    
                # Flatten the array to 1D and remove NaN values
                arr_flat = arr.flatten()
                arr_flat = arr_flat[~np.isnan(arr_flat)]
    
                if len(arr_flat) > 0:
                    # Calculate statistics: min, max, 25th, 50th, and 75th percentiles
                    min_val = np.min(arr_flat)
                    max_val = np.max(arr_flat)
                    pct_25 = np.percentile(arr_flat, 25)
                    pct_50 = np.percentile(arr_flat, 50)  # Median
                    pct_75 = np.percentile(arr_flat, 75)
                    pct_90 = np.percentile(arr_flat, 90)
                    print(min_val,
                         max_val,
                         pct_90 )
                             
                    # Create an Excel workbook and write the data
                    # Add the suffix to the Excel filename
                    output_excel_path = f"{output_directory}/raster_values_{point_id}_{suffix}.xlsx"
                    wb = Workbook()
    
                    # Write statistics to the first sheet, with a suffix in the sheet name
                    ws_stats = wb.active
                    ws_stats.title = f"Statistics_{suffix}"
                    ws_stats.append(["Statistic", "Value"])
                    ws_stats.append(["Min", min_val])
                    ws_stats.append(["Max", max_val])
                    ws_stats.append(["25th Percentile", pct_25])
                    ws_stats.append(["50th Percentile (Median)", pct_50])
                    ws_stats.append(["75th Percentile", pct_75])
                    ws_stats.append(["90th Percentile", pct_90])
    
                    # Create a second sheet for the raster values, with a suffix in the sheet name
                    ws_values = wb.create_sheet(title=f"All Values_{suffix}")
                    ws_values.append(["Raster Cell Values"])
                    for value in arr_flat:
                        ws_values.append([value])
    
                    # Save the Excel file with the appropriate suffix
                    wb.save(output_excel_path)
                    
    
                    print(f"Exported raster statistics and values for point ID {point_id} to {output_excel_path}")
                else:
                    print(f"No valid raster values for point ID {point_id}")
                    
            except:
                print("No data",point_geom)
    

24_19.tif
Exported clipped raster for point ID T16 to \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\Selected_Buffer\clipped_raster_T16_24_19.tif
-0.44099998 0.65800095 0.10400009155273438
Exported raster statistics and values for point ID T16 to \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\Selected_Buffer/raster_values_T16_24_19.xlsx
No data <geoprocessing describe geometry object object at 0x000002797C074510>
24_23.tif
Exported clipped raster for point ID T16 to \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\Selected_Buffer\clipped_raster_T16_24_23.tif
-0.29700074 0.32600036 0.05799880623817444
Exported raster statistics and values for point ID T16 to \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\Selected_Buffer/raster_values_T16_24_23.xlsx
No data <geoprocessing describe geometry object obje

# #  Delivery 3rd - P99, P95, P90, Mean, Median, STDEV,  ABS

In [8]:
import arcpy
import numpy as np
import os
import pandas as pd
from openpyxl import Workbook

# Set environment
arcpy.env.overwriteOutput = True
workspace = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin"
arcpy.env.workspace = workspace

# Input rasters (update file paths once 2024 data is available)
raster_2019 = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\SITEINV\20240206_from_WTP_004502963-03\004502963-03-MASP - GIS Data - Task 1 - Seabed Morphology and Mobility\RWE_Thor_Task1_v0.3.gdb\SN2019_011_R_DTM_MBES_100CM"
raster_2020 = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\SITEINV\20240206_from_WTP_004502963-03\004502963-03-MASP - GIS Data - Task 1 - Seabed Morphology and Mobility\RWE_Thor_Task1_v0.3.gdb\MMT_628_GU_OWF_T50R_DTU15_MSL_DTM_100CM"
raster_2023 = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\SITEINV\SITEINV.gdb\DK_THO_Fugro_MBES_2023_DEM_Mean_1pt00m_ra_UTM32N_EXT"
raster_2024 = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\SITEINV\20240924_Bathy\F248460_RWE_Thor_1m_MSL\F248460_RWE_Thor_1m_MSL.tif"

# Create a feature layer from the feature class
buffer_shapefile = r'\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\WTG_Buffer_200m.shp'
arcpy.MakeFeatureLayer_management(buffer_shapefile, "buffer_layer")


raster_pairs = [
    (raster_2024, raster_2019, "2024-2019", "24_19.tif", 0),     # 2024 - 2019
    (raster_2024, raster_2023, "2024-2023", "24_23.tif", -0.09), # 2024 - 2023
    (raster_2023, raster_2019, "2023-2019", "23_19.tif", 0.11),  # 2023 - 2019 (+0.11)
    (raster_2023, raster_2020, "2023-2020", "23_20.tif", 0.09),  # 2023 - 2020 (+0.09)
    (raster_2020, raster_2019, "2020-2019", "20_19.tif", 0)      # 2020 - 2019 (no correction)
]


# Output directory for Excel files and clipped raster exports
output_directory = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\20241017_Delivery_3rd"


# If the directory does not exist, create it
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    print(f"Directory '{directory}' created.")
    

    
# Create Excel workbook and sheet
wb = Workbook()
ws = wb.active
ws.title = "Statistics"
header_written = False
all_data = {}

In [ ]:
# If the directory does not exist, create it
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    print(f"Directory '{directory}' created.")
    
# Dictionary to store results for each point
all_data = {}

# Iterate through each raster pair
for raster1, raster2, years, output_name, correction in raster_pairs:
    print(f"Processing: {output_name}")

    # Iterate through each point to extract values from the difference raster
    with arcpy.da.SearchCursor("buffer_layer", ["Layout_ID1", "SHAPE@"]) as cursor:
        for row in cursor:
            point_id = row[0]  # Assuming 'Layout_ID1' contains the point ID
            point_geom = row[1]  # SHAPE@ returns the geometry object of the point

            try:
                # Clip the difference raster using the point geometry
                clipped_raster = arcpy.sa.ExtractByMask(output_name, point_geom)

                print(f"Point ID {point_id}")

                # Convert the clipped raster to a NumPy array
                arr = arcpy.RasterToNumPyArray(clipped_raster, nodata_to_value=np.nan)

                # Flatten the array and remove NaN values
                arr_flat = arr.flatten()
                arr_flat = arr_flat[~np.isnan(arr_flat)]

                if len(arr_flat) > 0:
                    # Calculate statistics and round to 2 decimal places
                    min_val = round(np.min(arr_flat), 2)
                    max_val = round(np.max(arr_flat), 2)
                    min_abs_val = round(np.min(np.abs(arr_flat)), 2)
                    max_abs_val = round(np.max(np.abs(arr_flat)), 2)

                    # Real values
                    pct_90 = round(np.percentile(arr_flat, 90), 2)
                    pct_95 = round(np.percentile(arr_flat, 95), 2)
                    pct_99 = round(np.percentile(arr_flat, 99), 2)
                    mean_value = round(np.mean(arr_flat), 2)
                    median_value = round(np.median(arr_flat), 2)
                    stdev_value = round(np.std(arr_flat), 2)

                    # Absolute values
                    pct_90_abs = round(np.percentile(np.abs(arr_flat), 90), 2)
                    pct_95_abs = round(np.percentile(np.abs(arr_flat), 95), 2)
                    pct_99_abs = round(np.percentile(np.abs(arr_flat), 99), 2)
                    mean_abs = round(np.mean(np.abs(arr_flat)), 2)
                    median_abs = round(np.median(np.abs(arr_flat)), 2)
                    stdev_abs = round(np.std(np.abs(arr_flat)), 2)

                    count_value = len(arr_flat)

                    # Store the statistics for this point and raster year
                    if point_id not in all_data:
                        all_data[point_id] = {}

                    all_data[point_id][years] = {
                        'Min': min_val,
                        'Max': max_val,
                        'Min ABS': min_abs_val,
                        'Max ABS': max_abs_val,
                        '90th PCT': pct_90,
                        '90th PCT ABS': pct_90_abs,
                        '95th PCT': pct_95,
                        '95th PCT ABS': pct_95_abs,
                        '99th PCT': pct_99,
                        '99th PCT ABS': pct_99_abs,
                        'Mean': mean_value,
                        'Mean ABS': mean_abs,
                        'Median': median_value,
                        'Median ABS': median_abs,
                        'STDEV': stdev_value,
                        'STDEV ABS': stdev_abs,
                        'Count': count_value
                    }
                else:
                    print(f"No valid raster values for point ID {point_id}")

            except arcpy.ExecuteError as e:
                if "010568" in str(e):
                    print(f"Invalid extent for point ID {point_id}. Skipping.")
                else:
                    print(f"Error processing point ID {point_id}: {str(e)}")
            except Exception as e:
                print(f"Error processing point ID {point_id}: {str(e)}")

# Write the header row only once, dynamically handling year-related columns
if not header_written:
    # Define the common headers
    base_headers = ["Point ID"]

    # Add yearly headers dynamically
    for year in raster_pairs:
        base_headers.extend([
            f"Min ({year[2]})", f"Max ({year[2]})", 
            f"90th PCT ({year[2]})", f"95th PCT ({year[2]})", f"99th PCT ({year[2]})",
            f"Mean ({year[2]})", f"Median ({year[2]})", f"STDEV ({year[2]})",
            f"Min ABS ({year[2]})", f"Max ABS ({year[2]})",
            f"90th PCT ABS ({year[2]})", f"95th PCT ABS ({year[2]})", f"99th PCT ABS ({year[2]})",
            f"Mean ABS ({year[2]})", f"Median ABS ({year[2]})", f"STDEV ABS ({year[2]})",
            f"Count ({year[2]})"
        ])

    # Write the header
    ws.append(base_headers)
    header_written = True

# Write data for each point in rows, but iterating over years horizontally
for point_id, years_data in all_data.items():
    row = [point_id]  # Start the row with the point ID
    for year, stats in years_data.items():
        row.extend([
            stats['Min'], stats['Max'],
            stats['90th PCT'], stats['95th PCT'], stats['99th PCT'],
            stats['Mean'], stats['Median'], stats['STDEV'],
            stats['Min ABS'], stats['Max ABS'],
            stats['90th PCT ABS'], stats['95th PCT ABS'], stats['99th PCT ABS'],
            stats['Mean ABS'], stats['Median ABS'], stats['STDEV ABS'],
            stats['Count']
        ])
    
    # Write the complete row for the point
    ws.append(row)

# Save the Excel file
output_excel_path = os.path.join(output_directory, "All_Points_Statistics.xlsx")
wb.save(output_excel_path)

print(f"Exported statistics for all points to {output_excel_path}")


# Create Excel workbook and sheet
wb = Workbook()
ws = wb.active
ws.title = "Statistics"

# Initialize a header list to avoid duplicate headers
header_written = False

# Write the header row only once, dynamically handling year-related columns
if not header_written:
    # Define the common headers
    base_headers = ["Point ID"]

    # Add yearly headers dynamically, but only for years that have data in all_data
    for year in raster_pairs:
        if any(year[2] in years_data for years_data in all_data.values()):  # Check if this year has data
            base_headers.extend([
                f"Min ({year[2]})", f"Max ({year[2]})", 
                f"90th PCT ({year[2]})", f"95th PCT ({year[2]})", f"99th PCT ({year[2]})",
                f"Mean ({year[2]})", f"Median ({year[2]})", f"STDEV ({year[2]})",
                f"Min ABS ({year[2]})", f"Max ABS ({year[2]})",
                f"90th PCT ABS ({year[2]})", f"95th PCT ABS ({year[2]})", f"99th PCT ABS ({year[2]})",
                f"Mean ABS ({year[2]})", f"Median ABS ({year[2]})", f"STDEV ABS ({year[2]})",
                f"Count ({year[2]})"
            ])

    # Write the header
    ws.append(base_headers)
    header_written = True

# Write data for each point in rows, iterating over years that have data
for point_id, years_data in all_data.items():
    row = [point_id]  # Start the row with the point ID
    
    # Iterate through the years in raster_pairs
    for year in raster_pairs:
        year_key = year[2]  # Extract the year from raster_pairs

        if year_key in years_data:  # Check if the year has data for this point
            stats = years_data[year_key]
            row.extend([
                stats['Min'], stats['Max'],
                stats['90th PCT'], stats['95th PCT'], stats['99th PCT'],
                stats['Mean'], stats['Median'], stats['STDEV'],
                stats['Min ABS'], stats['Max ABS'],
                stats['90th PCT ABS'], stats['95th PCT ABS'], stats['99th PCT ABS'],
                stats['Mean ABS'], stats['Median ABS'], stats['STDEV ABS'],
                stats['Count']
            ])
        else:
            # If no data for this year, append empty cells to maintain consistency
            row.extend([""] * 17)  # 17 empty fields for missing year stats

    # Write the complete row for the point
    ws.append(row)

# Save the Excel file
output_excel_path = os.path.join(output_directory, "All_Points_Statistics.xlsx")
wb.save(output_excel_path)

print(f"Exported statistics for all points to {output_excel_path}")



Processing: 24_19.tif
Invalid extent for point ID T01. Skipping.
Invalid extent for point ID T02. Skipping.
Invalid extent for point ID T03. Skipping.
Invalid extent for point ID T04. Skipping.
Invalid extent for point ID T05. Skipping.
Point ID T06
No valid raster values for point ID T06
Point ID T07
Invalid extent for point ID T08. Skipping.
Invalid extent for point ID T09. Skipping.
Invalid extent for point ID T10. Skipping.
Invalid extent for point ID T11. Skipping.
Invalid extent for point ID T12. Skipping.
Point ID T13
Invalid extent for point ID T14. Skipping.
Invalid extent for point ID T15. Skipping.
Point ID T16
